# Predicting survival on the Titanic:
# An introduction to Machine Learning with Python
*By Alexander I.R Jackson, University of Southampton*

In this notebook we will work with data about passengers on the RMS Titanic's maiden voyage; which famously ended in disaster.

We'll use python and machine learning libraries to try and understand and predict survival on this fateful voyage.

## Setup

### Load Packages
Here we will load all the packages needed for our analysis. You can modify this list and add to it if you want.


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier


### Import the data
We will use Pandas to import the data from 'titanic.csv' file and store it in a dataframe

In [ ]:
# Download the data from GitHub
! mkdir data
! curl https://github.com/SandyJackson/clinai-ssu-titanic/blob/95a23e83e7dbe3b4b09f6594286019ae62b86e18/data/titanic.csv --output data/titanic.csv

In [ ]:
# This line reads the data and stores the information in the titanic_df variable
titanic_df = pd.read_csv('data/titanic.csv')
titanic_df

## Explore the data




### Getting an overview
Pandas DataFrames have a number of useful methods to explore DataFrames. 
We use these by writing the variable name (`titanic_df`) then a dot (`.`) and then the method like `head(x)` - which shows the x first rows.
Some other methods you can use include:
* `tail()`
* `sample()`
* `shape` - notice this isn't actually a method, there is no () instead it's an attribute or property.
* `info()`
* `describe()`

Try these out and see if you can work out what they do. We've done an example with `head()`.

In [ ]:
# Head
titanic_df.head(10)

In [ ]:
# Tail

In [ ]:
# Sample


In [ ]:
# Shape

In [ ]:
# Info

In [ ]:
# Describe


### Data understanding
Some of these columns don't make a lot of sense! Luckily we've done the hard work and found what the different random numbers and letters mean. In the real world this can be much harder
```markdown
Variable    Definition      Key

survival    Survival        0 = No, 1 = Yes
pclass      Ticket class    1 = 1st, 2 = 2nd, 3 = 3rd
name        Passenger Name
sex         Sex
Age         Age in years
sibsp       # of siblings / spouses aboard the Titanic
parch       # of parents / children aboard the Titanic
ticket      Ticket number
fare        Passenger fare
cabin       Cabin number
embarked    Port of Embarkation C = Cherbourg, Q = Queenstown, S = Southampton
```

### Select a column
If you just want to select one column there are a few ways.
The simplest is using square brackets `[]` and the columns name in quotation marks `''` or `""`

for example:
`titanic_df['name']`

Try this out below!

P.S. If you are really getting into this then it's worth reading [the Pandas documentation on this](https://pandas.pydata.org/docs/user_guide/indexing.html) as there are different ways to select and index data, which should be used at different times!

In [ ]:
# Select a column by name

### Getting some insights

Now you can select a column lets try and understand something about the data...

These methods can be used on individual columns (called Series in Pandas) and can be useful in lots of ways

* `mean()`
* `median()`
* `min()`, `max()`
* `value_counts()`

Try some out below. Remember it's hard to calculate mean on a name! (or any string) and `value_counts()` probably isn't the best way to summarise the fare.

In [ ]:
# Try it out below

In [ ]:
# Where did most passengers board the Titanic?

In [ ]:
# This section is a bit more complicated... can you figure it out?

titanic_df.groupby('pclass').agg(mean_fare=('fare', 'mean'),
                                 mean_age=('age', 'mean'),
                                 count=('pclass', 'count'))

## Tidy the data

This is probably the most time consuming and most important part of machine learning and data science. Don't worry though... not today!

The Titanic dataset is already pretty clean and well structured, but there are still a few things to think about


### Selecting the important columns

Not all the columns will be useful for predicting who will survive. Some columns are best left out of the modelling stage.

Can you think which ones and get rid of them using the code below?

<details>
    <summary markdown='span'>💡 Hint</summary>

Anything that is unique to each individual is unlikely to be useful in predicting outcome (unless you do some additional clever analysis). There are two 'unique' columns
</details>

In [ ]:
# The code below will 'drop' the columns that you specify.
# Which columns should we leave out? (Remember if you want to drop more than one its a list of strings)
# Notice we save the output to a new DataFrame called titanic_df_clean, we'll use this going forward

titanic_df_clean = titanic_df.drop(columns=['COL NAME HERE'], axis=1)

# Now print that new DataFrame out. Does it look right?
titanic_df_clean

### The Cabin Column

Let's have a close look at this column.


In [ ]:
# Take a random sample of 10 rows from the DataFrame
titanic_df['cabin'].sample(10, random_state=42)

There seem to be a lot of NaN (missing values). Let's try and work out why.

* What does the column mean?
* Why are there so many missing values?
* What could we do to make it useful?

<details>
    <summary markdown='span'>💡 Hint</summary>

Maybe only passengers who could afford a cabin will have one? Or maybe the data are just poor?
</details>

Below we'll try one possible approach to make the cabin data more useful

In [ ]:
# Make a blank list to store our new columns data in
cabin_bool = []

# Now a for loop goes through every row in the 'cabin' column
# Can you work out what it does?
for cabin in titanic_df['cabin']:
    if pd.isna(cabin):
        cabin_bool.append(False)
    else:
        cabin_bool.append(True)

# Print out the first 10 values of the new list to see what it does
cabin_bool[0:10]

<details>
    <summary markdown='span'>💡 Hint (if the loop doesn't make sense)</summary>
the pd.isna() function checks if a value is missing (NaN)
</details>

Remember in programming there are often lots of ways to achieve the same thing

In [ ]:
# The line below does exactly the same thing but in one line

cabin_bool_quick = [True if not pd.isna(cabin) else False for cabin in titanic_df_clean['cabin']]

# Lets check they are the same (should print True if they are the same)
cabin_bool == cabin_bool_quick


In [ ]:
# Create a new 'cabin_bool' column in the DataFrame
titanic_df_clean['cabin_bool'] = cabin_bool

# Drop the old cabin column
titanic_df_clean.drop(columns=['cabin'], axis=1, inplace=True)

# Check the new DataFrame
titanic_df_clean

### The missing values
We know our data have some missing values - you might have noticed in the early exploration (`.info()`)

In [ ]:
# But just how many missing values are there?
titanic_df_clean.isna().sum()

`age` seems to have the most missing values (>250)

**What are our options?**
* Should we get rid of age completely?
    * Remember that 'women and children go first' was important on the titanic, so age might be important
* Should we just get rid of the rows where age is missing?
* What else could we do?

<details>
    <summary markdown='span'>💡 Hint</summary>
We could put a value in or 'impute'.
This could be the mean, median or any other value we like</details>

**What about the other few missing values in other colums?**

To keep things simple, we could just select only columns with all the data complete

In [ ]:
# This line will drop all rows with missing values
titanic_df_clean.dropna()


In [ ]:
# How many rows did we lose?
titanic_df_clean.shape[0] - titanic_df_clean.dropna().shape[0]

In [ ]:
# Lets just use the rows without missing values for now!
titanic_df_clean = titanic_df_clean.dropna()

## Let's start modelling!

We'll now go on and try some simple statistical modelling before building up to more complex machine learning.

Remember our goal is to predict which passengers will survive.



### Splitting the data

But first up we need a training set and a test set. We also need our X (inputs) and our y (outcome) separate.

Luckily there are packages that will do a lot of the legwork for you. Below we use the `train_test_split` function from sklearn ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html))

You'll still need to select the correct y (the outcome we are interested in)

<details>
    <summary markdown='span'>💡 Hint </summary>
Remember we want to predict survival and the colum is called survived. So try selecting that column using the square brackets []
</details>

In [ ]:
# Select our X
X = titanic_df_clean.drop(columns='survived', axis=1)

# Select our y
y = # YOUR ANSWER HERE

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

In [ ]:
# Try comparing the size of the training and testing sets (remember the shape attribute)



## Logisitic Regression
This will be our most simple model. You'll see Logistic regression used in lots of medical publications - its really common!
Some people wouldn't even regard it as 'Machine Learning' but all the same basic principles apply

In [ ]:
# This creates a model object that we can train with our data
model = LogisticRegression(penalty='none')

# This line will train and test our model (using only the training data)
cross_validate(model, X_train, y_train, cv=5, scoring='accuracy')

# Does it work?

**Failure**

But can you work out why?

The error message says:
`ValueError: could not convert string to float: 'male'`

Remember computers can only understand numbers

### Continuous vs Categorical
First off we need to work out which columns are categorical and which are continuous

In [ ]:
# Create a list of each type of column
categorical_columns = ['sex', 'embarked', 'cabin_bool']
numerical_columns = ['pclass', 'age', 'sibsp', 'parch', 'fare']

In [ ]:
# Lets try to model using only the numerical columns
model = LogisticRegression(penalty=None, max_iter=5000)
cross_validate(model, X_train[numerical_columns], y_train, cv=5, scoring='accuracy')

### Encoding Categorical Data

Some of those categorical variables may be very helpful in predicting survival.

So let's take all of our categorical data and encode it into a format that our model can understand.

First off which variables are categorical? Put them into the list below

In [ ]:
# Create an object to encode the categorical data; we'll use sklearn's OneHotEncoder (don't worry too much about specifics)
ohe = OneHotEncoder(drop='if_binary', sparse_output=False, handle_unknown='ignore')

# Fit the encoder to the training data (i.e. learn the categories)
ohe.fit(X_train[categorical_columns])



In [ ]:
# Now we transform those cataegorical columns into numbers
encoded_cols = ohe.transform(X_train[categorical_columns])

# Print the values out
print(f"The encoded values have {encoded_cols.shape[0]} rows and {encoded_cols.shape[1]} columns")
print('But what do they represent?')
encoded_cols

In [ ]:
# We don't know what they mean. Luckily, we can ask the encoder to tell us!
ohe.get_feature_names_out()

In [ ]:
# Now lets put those together and make a new dataframe with encoded values
# Lets start by taking just the numerical columns
X_train_encoded = X_train[numerical_columns].copy()
# Let's see what we get
X_train_encoded


In [ ]:
# Now lets add the encoded categorical columns, we get the names from the encoder and then the data from the unnamed array above
X_train_encoded[ohe.get_feature_names_out()] = encoded_cols

# Lets print it out and see what it looks like
X_train_encoded

In [ ]:
# Lets look back at the data before 'encoding' and see if it makes sense.
# Compare the two (new is above and old is below) and see if you can understand what is happening with encoding
X_train

In [ ]:
# Now lets see if all our work has paid off!
# We'll try out the model on our training data and see how it performs
cross_validate(model, X_train_encoded, y_train, cv=5, scoring='accuracy')

### Testing the model

OK so we have a model with all our data that seems to work. Shall we test it the right way?

* What is the right way?
* Why is using the `test` data only for testing important?

In [ ]:
# We'll train the model using only the training data using .fit
model.fit(X_train_encoded, y_train)

# Lets see how the model performs and save the score
train_score = model.score(X_train_encoded, y_train)


#### Prepare the test data
Remember all that work to encode the categorical variables?
We have to do it again, but this time it's much quicker

In [ ]:
# See if you can work out what the code below does
X_test_encoded = X_test[numerical_columns].copy()
X_test_encoded[ohe.get_feature_names_out()] = ohe.transform(X_test[categorical_columns])
print("Is it the same format as the training data?")
X_test_encoded


#### Let's get the score!

In [ ]:
# We use the same model, but this time we score it on the test data
test_score = model.score(X_test_encoded, y_test)

print(f"Train score: {train_score}")
print(f"Test score: {test_score}")

* Does the model perform better or worse?
* Why might that be?

### Other Model Types (Optional)

Logisitc regression is pretty common - it's also quite simple. But for this task it still performs pretty well.

However, there are lots of other machine learning models you can play about with. Below we will try some out.

But first more data manipulation

#### Scaling the numerical columns
Basic logistic regression works with any old numbers.
Unfortunately, most machine learning methods (even more advanced logistic regression) require something called scaling

Basically all the different variables need to be a similar scale or the model might incorrectly think the biggest number is the most important!

* What would that mean for the Titanic data we already have?

OK let's scale these numbers!

In [ ]:
# The scaler is a bit like the encoder, but it scales the numerical data (this one scales between 0 and 1)
scaler = MinMaxScaler()

# We fit and transform just like before
scaler.fit(X_train[numerical_columns])
scaled_data = scaler.transform(X_train[numerical_columns])

# Now we put it all together
X_train_scaled = pd.DataFrame(scaled_data, columns=numerical_columns)
X_train_scaled[ohe.get_feature_names_out()] = encoded_cols
X_train_scaled

In [ ]:
# We do the same for the test data
X_test_scaled = pd.DataFrame(scaler.transform(X_test[numerical_columns]), columns=numerical_columns)
X_test_scaled[ohe.get_feature_names_out()] = ohe.transform(X_test[categorical_columns])
X_test_scaled

#### Trying some models
We'll try out 3 different models and see how they perform

These are working on default settings and may perform well, they may perform poorly and they might make some incorrect assumptions

**This is not best practice but we are showing you how quickly you can try out different models**

In [ ]:
# A multinomial naive bayes model
bayes_model = MultinomialNB()
cross_validate(bayes_model, X_train_scaled, y_train, cv=5, scoring='accuracy')

In [ ]:
# A support vector classifier
svm_model = SVC()
cross_validate(svm_model, X_train_scaled, y_train, cv=5, scoring='accuracy')


In [ ]:
# A gradient boosting classifier
gb_model = GradientBoostingClassifier()
cross_validate(gb_model, X_train_scaled, y_train, cv=5, scoring='accuracy')


#### Hyperparameter tuning
The support vector classifier and gradient boosting classifer both seem to perform quite well; but the gradient boosting is performing slightly better.

Let's see if we can improve it with some tuning

We'll start by getting some baseline results

In [ ]:
# Fit the model 
gb_model.fit(X_train_scaled, y_train)

# Calculate the training and testing scores, and print them below
print("Gradient Boosting:")
print(f"Training score: {gb_model.score(X_train_scaled, y_train)}")
print(f"Testing score: {gb_model.score(X_test_scaled, y_test)}")

So it performs very well on the training data but less well on the testing.
* What is this a sign of?
<details>
    <summary markdown='span'>💡 Hint</summary>
Overfitting
</details>

So let's see if we can improve test performance with hyperparameter tuning

**Grid Search**

We'll do something called a `grid search`. Here we give a list of different parameters of the model we can adjust to a function.

The function then tries every possible combination on the training data using something called cross-validation.

Once it's finished it can tell you the combination that performs best.

For more information about the parameters you can look at the [Gradient Boosting Classifier documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)

To learn more about hyperparameter tuning and different kinds of searches (grid, random etc.) look at [this sklearn guide](https://scikit-learn.org/stable/modules/grid_search.html#grid-search)

In [ ]:
# Create the dictionary of parameters
param_grid = {'n_estimators': [50, 100, 200],
                'max_depth': [3, 5, 7],
                'learning_rate': [0.1, 0.01, 0.001]}

# Setup the grid search with our model, the parameter grid and a few other variables
grid_search = GridSearchCV(
    estimator= gb_model, 
    param_grid = param_grid, 
    cv=5, scoring='accuracy')

# Now we run the grid search. This will take a while!
grid_search.fit(X_train_scaled, y_train)

In [ ]:
# Now we have done the .fit we can see the best parameters
print(grid_search.best_params_)
# And the best score (notice this time we make it a percentage for readability!)
print(f"Grid Search best score: {round(grid_search.best_score_*100, 2)}% accurate")

In [ ]:
# Now we instantiate a new model with the best parameters
gb_model_optimised = GradientBoostingClassifier(**grid_search.best_params_)
# Lets check if it has the 'best parameters'
gb_model_optimised

In [ ]:
# Now we train the new model on our training data
gb_model_optimised.fit(X_train_scaled, y_train)

# Finally we can print the training and testing scores
print("Gradient Boosting:")
print(f"Training score: {gb_model_optimised.score(X_train_scaled, y_train)}")
print(f"Testing score: {gb_model_optimised.score(X_test_scaled, y_test)}")

# Fin
Well done you've reached the end.

There is lot's more we can talk about or do and load more to learn.

You could try optimising some different models; or maybe you want to try some different approaches to the data.
* Should `Pclass` be ordinal rather than numeric?
* Could we impute the 250 missing `age` to make our dataset bigger?
* Could we drop some of the variables that aren't making any difference?